In [ ]:
import requests
import random
from datetime import datetime, timedelta
import pandas as pd

In [15]:
# Generate random dates for testing
start_date = datetime(2010, 1, 1)
end_date = datetime(2024, 12, 31)
date_tests = []

while len(date_tests) < 15:
    random_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
    if random_date.weekday() < 5:  # Monday to Friday are 0-4
        date_tests.append(random_date.strftime('%Y-%m-%d'))

date_tests

['2013-07-12',
 '2013-09-02',
 '2012-08-09',
 '2012-10-24',
 '2011-07-13',
 '2010-07-22',
 '2019-01-22',
 '2015-04-08',
 '2018-01-03',
 '2016-11-28',
 '2013-11-26',
 '2015-01-13',
 '2014-02-10',
 '2015-04-29',
 '2010-04-29']

In [ ]:
predicted_close_price = []

for date in date_tests:
    response = requests.post(f"http://localhost:5000/api/predict", json={"date": date})
    predicted_close_price.append(response.json().get("predicted_close_price"))

predicted_close_price

[22.826265335083008,
 21.454792022705078,
 16.129493713378906,
 17.224336624145508,
 13.274055480957031,
 11.705143928527832,
 54.841697692871094,
 27.460172653198242,
 52.95281219482422,
 39.27250671386719,
 25.999860763549805,
 25.355812072753906,
 29.105066299438477,
 28.483688354492188,
 13.52993392944336]

In [20]:

# Load the CSV file into a dataframe
csv_file_path = 'data_GOOGL_2010-2024.csv'
df = pd.read_csv(csv_file_path)

# Convert the 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Filter the dataframe for the dates in date_tests
filtered_df = df[df['Date'].isin(pd.to_datetime(date_tests))]

# Compare the close price with the predicted close price
comparison = []
for date, predicted in zip(date_tests, predicted_close_price):
    actual_close = filtered_df.loc[filtered_df['Date'] == date, 'Close'].values
    if actual_close.size > 0:
        difference = abs(actual_close[0] - predicted)
        comparison.append((date, actual_close[0], predicted, difference))

comparison_df = pd.DataFrame(comparison, columns=['Date', 'Actual Close', 'Predicted Close', 'Difference'])
print(comparison_df)

          Date  Actual Close  Predicted Close  Difference
0   2013-07-12     23.014669        22.826265    0.188404
1   2012-08-09     16.016764        16.129494    0.112730
2   2012-10-24     16.888227        17.224337    0.336109
3   2011-07-13     13.421317        13.274055    0.147262
4   2010-07-22     12.088560        11.705144    0.383416
5   2019-01-22     53.736698        54.841698    1.105000
6   2015-04-08     27.342878        27.460173    0.117294
7   2018-01-03     54.378876        52.952812    1.426064
8   2016-11-28     39.147591        39.272507    0.124916
9   2013-11-26     26.391068        25.999861    0.391207
10  2015-01-13     24.999374        25.355812    0.356438
11  2014-02-10     29.246582        29.105066    0.141516
12  2015-04-29     27.968115        28.483688    0.515574
13  2010-04-29     13.265225        13.529934    0.264709
